In [3]:
import torch

In [4]:
full_dataset = torch.load("/fs/pool/pool-marsot/bindbind/datasets/tankbind_processed/processed/data.pt")

In [5]:
training_indices_path = "/fs/pool/pool-marsot/tankbind_enzo/bind/packages/EquiBind/data/timesplit_no_lig_overlap_train"

In [6]:
with open(training_indices_path, "r") as f:
    train_names = set([name for name in f.read().split("\n")])

In [9]:
train_dataset = {name: full_dataset[name] for name in train_names if name in full_dataset}

In [13]:
train_dataset['6d8e'].keys()

dict_keys(['diffdock_ligand_atom_features', 'diffdock_ligand_atom_coordinates', 'diffdock_ligand_edge_index', 'diffdock_ligand_edge_attr', 'diffdock_protein_sequences', 'diffdock_protein_node_features', 'diffdock_protein_chain_ids', 'diffdock_protein_alpha_carbon_coordinates', 'diffdock_protein_residue_atom_coordinates', 'diffdock_protein_sidechain_vector_features', 'diffdock_protein_alpha_carbon_knn_edge_index', 'tankbind_ligand_atom_coordinates', 'tankbind_ligand_atom_features', 'tankbind_ligand_edge_index', 'tankbind_ligand_edge_features', 'tankbind_ligand_pairwise_distance_distribution', 'tankbind_protein_alpha_carbon_coordinates', 'tankbind_protein_node_scalar_features', 'tankbind_protein_node_vector_features', 'tankbind_protein_edge_index', 'tankbind_protein_edge_scalar_features', 'tankbind_protein_edge_vector_features', 'tankbind_one_letter_sequence', 'protein_name', 'tankbind_compound_center', 'tankbind_num_protein_nodes_close_to_ligand_and_in_contact_with_ligand', 'tankbind_pr

In [17]:
proteins_csv = torch.load("/fs/pool/pool-marsot/bindbind/datasets/tankbind_processed/processed/proteins_csv.pt")

In [22]:
proteins_csv[proteins_csv["protein_names"]=="2lbv"]["affinity"].values[0]

7.59

In [23]:
list_protein_node_scalar_features = []
list_protein_node_vector_features = []
list_protein_edge_scalar_features = []
list_protein_edge_vector_features = []
list_protein_distance_to_compound = []
list_affinity = []
for name, item in train_dataset.items():
    # Input features
    list_protein_node_scalar_features.append(item["tankbind_protein_node_scalar_features"])
    list_protein_node_vector_features.append(item["tankbind_protein_node_vector_features"])
    list_protein_edge_scalar_features.append(item["tankbind_protein_edge_scalar_features"])
    list_protein_edge_vector_features.append(item["tankbind_protein_edge_vector_features"])
    distance = torch.cdist(item["tankbind_protein_alpha_carbon_coordinates"], item["tankbind_ligand_atom_coordinates"])
    distance = distance[torch.any(distance<20.0, dim=1)].clamp(0, 10).flatten()
    # don't add other input features because they are onehot
    # targets
    list_protein_distance_to_compound.append(distance)
    affinity = proteins_csv[proteins_csv["protein_names"]==name]["affinity"].values[0]
    list_affinity.append(affinity)

In [26]:
print(torch.var_mean(torch.cat(list_protein_node_scalar_features, dim=0), dim=0))
print(torch.var_mean(torch.cat(list_protein_node_vector_features, dim=0), dim=0))
print(torch.var_mean(torch.cat(list_protein_edge_scalar_features, dim=0), dim=0))
print(torch.var_mean(torch.cat(list_protein_edge_vector_features, dim=0), dim=0))
print(torch.var_mean(torch.cat(list_protein_distance_to_compound, dim=0), dim=0))
print(torch.var_mean(torch.tensor(list_affinity), dim=0))

(tensor([0.2220, 0.6001, 0.0291, 0.1942, 0.3817, 0.0095]), tensor([ 0.0727,  0.1325, -0.9804, -0.7606,  0.0249,  0.0119]))
(tensor([[0.3323, 0.3322, 0.3328],
        [0.3323, 0.3322, 0.3328],
        [0.3335, 0.3342, 0.3322]]), tensor([[-0.0006,  0.0006,  0.0004],
        [ 0.0006, -0.0006, -0.0004],
        [-0.0001,  0.0010, -0.0002]]))


KeyboardInterrupt: 

In [27]:
means_dict = {"protein_node_scalar_features": torch.tensor([0.07262682914733887, 0.1329594999551773, -0.9804213643074036, -0.760931670665741, 0.02471655048429966, 0.011947994120419025]),
              "protein_node_vector_features": torch.tensor([[-0.0005944063304923475, 0.0004770602972712368, 0.0003047776408493519], [0.0005944063304923475, -0.0004770602972712368, -0.0003047776408493519], [-9.438615961698815e-05, 0.0009057653369382024, -0.00030406430596485734]]),
              "protein_edge_scalar_features": torch.tensor([6.944148026377661e-06, 0.001373058999888599, 0.03128135949373245, 0.10510130971670151, 0.1631690412759781, 0.17957991361618042, 0.2117183655500412, 0.3174160420894623, 0.30822935700416565, 0.18642766773700714, 0.09413681924343109, 0.039788469672203064, 0.014366302639245987, 0.005030795466154814, 0.001948254881426692, 0.0008591284276917577, -0.03676437586545944, -0.03928199037909508, -0.027324488386511803, -0.04502237215638161, -0.03382807597517967, -0.0309743732213974, -0.03358544036746025, -0.00877009890973568, -0.0006605424568988383, -0.0003721231478266418, 2.121787292708177e-05, 8.425789928878658e-06, 4.6160774218151346e-05, -9.526106623525266e-06, 0.0002609408984426409, 4.474521119846031e-05]),
              "protein_edge_vector_features": torch.tensor([[-0.0002645916829351336, -0.0002201295574195683, -3.9455248042941093e-05]]),
              "protein_distance_to_compound": torch.tensor(8.333386421203613),
              "compound_to_compound": torch.tensor([0.5834429264068604, 0.09144987910985947, 0.001274818554520607, 0.32383236289024353, 0.9850631952285767, 0.0, 0.0, 0.004669341258704662, 0.010267456993460655, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5256147384643555, 1.4313571453094482]),
              "affinity": torch.tensor(6.3677),
              "protein_distance_to_compound": torch.tensor(9.7262)
              }

In [29]:
print(list(means_dict.keys()))

['protein_node_scalar_features', 'protein_node_vector_features', 'protein_edge_scalar_features', 'protein_edge_vector_features', 'protein_distance_to_compound', 'compound_to_compound', 'affinity']


In [31]:
list_keys = ['protein_node_scalar_features', 'protein_node_vector_features', 'protein_edge_scalar_features', 'protein_edge_vector_features', 'protein_distance_to_compound', 'affinity']

In [32]:
var_protein_node_scalar_features, mean_protein_node_scalar_features = torch.var_mean(torch.cat(list_protein_node_scalar_features, dim=0), dim=0)
var_protein_node_vector_features, mean_protein_node_vector_features = torch.var_mean(torch.cat(list_protein_node_vector_features, dim=0), dim=0)
var_protein_edge_scalar_features, mean_protein_edge_scalar_features = torch.var_mean(torch.cat(list_protein_edge_scalar_features, dim=0), dim=0)
var_protein_edge_vector_features, mean_protein_edge_vector_features = torch.var_mean(torch.cat(list_protein_edge_vector_features, dim=0), dim=0)
var_protein_distance_to_compound, mean_protein_distance_to_compound = torch.var_mean(torch.cat(list_protein_distance_to_compound, dim=0), dim=0)
var_affinity, mean_affinity = torch.var_mean(torch.tensor(list_affinity), dim=0)
means_dict = {"protein_node_scalar_features": mean_protein_node_scalar_features,
                "protein_node_vector_features": mean_protein_node_vector_features,
                "protein_edge_scalar_features": mean_protein_edge_scalar_features,
                "protein_edge_vector_features": mean_protein_edge_vector_features,
                "protein_distance_to_compound": mean_protein_distance_to_compound,
                "affinity": mean_affinity}

vars_dict = {"protein_node_scalar_features": var_protein_node_scalar_features,
                "protein_node_vector_features": var_protein_node_vector_features,
                "protein_edge_scalar_features": var_protein_edge_scalar_features,
                "protein_edge_vector_features": var_protein_edge_vector_features,
                "protein_distance_to_compound": var_protein_distance_to_compound,
                "affinity": var_affinity}

stds_dict = {key: torch.sqrt(var) for key, var in vars_dict.items()}

In [35]:
torch.set_printoptions(precision=10)

In [36]:
print(means_dict)

{'protein_node_scalar_features': tensor([ 0.0727457255,  0.1325012892, -0.9804269075, -0.7606167793,
         0.0248864293,  0.0118933702]), 'protein_node_vector_features': tensor([[-0.0006267764,  0.0006361017,  0.0003718575],
        [ 0.0006267764, -0.0006361017, -0.0003718575],
        [-0.0001250555,  0.0010360993, -0.0001783383]]), 'protein_edge_scalar_features': tensor([ 6.9658167376e-06,  1.3754346874e-03,  3.1302969903e-02,
         1.0506523401e-01,  1.6300360858e-01,  1.7937441170e-01,
         2.1146914363e-01,  3.1705018878e-01,  3.0818232894e-01,
         1.8676058948e-01,  9.4478376210e-02,  4.0007371455e-02,
         1.4457522891e-02,  5.0572101027e-03,  1.9602947868e-03,
         8.6626538541e-04, -3.6794535816e-02,  9.7907505929e-02,
         2.7249491215e-01,  4.7467842698e-01,  7.6569545269e-01,
         9.5167309046e-01,  9.9420434237e-01,  9.9940609932e-01,
        -6.3369655982e-04,  2.6633599191e-04,  7.6850854384e-05,
         5.2463921020e-04, -1.2254845351e-0

In [37]:
print(stds_dict)

{'protein_node_scalar_features': tensor([0.4711185396, 0.7746803761, 0.1705850214, 0.4407012165, 0.6178143024,
        0.0975829586]), 'protein_node_vector_features': tensor([[0.5764503479, 0.5763314366, 0.5768814683],
        [0.5764503479, 0.5763314366, 0.5768814683],
        [0.5775108337, 0.5781271458, 0.5764107108]]), 'protein_edge_scalar_features': tensor([6.2377876020e-05, 5.5922851898e-03, 1.0898678005e-01, 2.6428866386e-01,
        3.0745354295e-01, 3.0576357245e-01, 3.0775597692e-01, 3.6643519998e-01,
        3.6639633775e-01, 3.0458340049e-01, 2.3440334201e-01, 1.5616969764e-01,
        9.3920931220e-02, 5.5855795741e-02, 3.5477731377e-02, 2.3793805391e-02,
        6.9635939598e-01, 6.8488472700e-01, 7.2912204266e-01, 6.6055721045e-01,
        4.5727527142e-01, 1.5990658104e-01, 2.7558168396e-02, 3.0823557172e-03,
        7.1674913168e-01, 7.2204363346e-01, 6.2779581547e-01, 5.8167368174e-01,
        4.5233646035e-01, 2.6219883561e-01, 1.0391493887e-01, 3.4320969135e-02]), '

: 